In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

!pip install torchattacks

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/models')


In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchattacks import FGSM, PGD, CW
from tqdm import tqdm

from wideresnet import WideResNet

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define WideResNet hyperparameters
depth = 28
width = 10
num_classes = 10
batch_size = 32  # Reduced batch size to minimize memory usage

# Define data transformations and load the dataset
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_dataset = datasets.CIFAR10(root='/content/data', train=False, download=True, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained WideResNet model
def load_wideresnet_model(model_path, depth, width, num_classes):
    model = WideResNet(depth=depth, width=width, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

# Replace this with the path to your pre-trained model weights
model_path = '/content/drive/MyDrive/best_wrn_model.pth'
model = load_wideresnet_model(model_path, depth, width, num_classes)

# Function to test the model's accuracy
def test_model_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc='Evaluating Clean Accuracy', leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Clear CUDA cache
torch.cuda.empty_cache()

# Test clean model accuracy
print('Model accuracy on clean test data:')
test_model_accuracy(model, test_loader)

# Attack Functions
def add_noise(images, noise_factor):
    noisy_images = images + noise_factor * torch.randn_like(images)
    return torch.clamp(noisy_images, 0, 1)

def attack_fgsm(model, images, labels, epsilon):
    images.requires_grad = True
    attack = FGSM(model, eps=epsilon)
    return attack(images, labels)

def attack_pgd(model, images, labels, epsilon, alpha, steps):
    images.requires_grad = True
    attack = PGD(model, eps=epsilon, alpha=alpha, steps=steps)
    return attack(images, labels)

def attack_cw(model, images, labels):
    images.requires_grad = True
    attack = CW(model, c=1e-4, kappa=0, steps=1000, lr=0.01)
    return attack(images, labels)

# Function to evaluate model on adversarial examples
def evaluate_adversarial_examples(model, data_loader, attack_fn, description='Attacking'):
    model.eval()
    correct = 0
    total = 0
    for images, labels in tqdm(data_loader, desc=description, leave=False):
        images, labels = images.to(device), labels.to(device)
        adv_images = attack_fn(model, images, labels)
        outputs = model(adv_images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Compare performance drops for different noise magnitudes
def compare_performance_drops(model, data_loader):
    noise_epsilons = [0.05, 0.1, 0.2]
    fgsm_epsilons = [0.05, 0.1, 0.2]
    pgd_epsilons = [0.05, 0.1, 0.2]
    num_steps = 40
    step_size = 0.01

    print('\nFGSM Attack:')
    for eps in tqdm(fgsm_epsilons, desc='FGSM Attack', leave=False):
        accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: attack_fgsm(model, x, y, epsilon=eps), description=f'FGSM 𝜖={eps}')
        print(f'FGSM 𝜖 = {eps}: Accuracy after attack: {accuracy:.2f}%')

    print('\nPGD Attack:')
    for eps in tqdm(pgd_epsilons, desc='PGD Attack', leave=False):
        accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: attack_pgd(model, x, y, epsilon=eps, alpha=step_size, steps=num_steps), description=f'PGD 𝜖={eps}')
        print(f'PGD 𝜖 = {eps}: Accuracy after attack: {accuracy:.2f}%')

    print('\nC-W Attack:')
    accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: attack_cw(model, x, y), description='C-W Attack')
    print(f'C-W attack: Accuracy after attack: {accuracy:.2f}%')

    print('\nNoise Attack:')
    for eps in tqdm(noise_epsilons, desc='Noise Attack', leave=False):
        accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: add_noise(x, eps), description=f'Noise 𝜖={eps}')
        print(f'Noise magnitude 𝜖 = {eps}: Accuracy after attack: {accuracy:.2f}%')

# Test performance drop comparison
compare_performance_drops(model, test_loader)


Files already downloaded and verified
Model accuracy on clean test data:


Accuracy: 79.48%

FGSM Attack:


FGSM Attack:  33%|███▎      | 1/3 [00:20<00:41, 20.57s/it]

FGSM 𝜖 = 0.05: Accuracy after attack: 11.97%



FGSM Attack:  67%|██████▋   | 2/3 [00:41<00:20, 20.54s/it]

...


PGD Attack:  67%|██████▋   | 2/3 [16:33<08:16, 496.86s/it]

PGD 𝜖 = 0.1: Accuracy after attack: 0.15%



PGD 𝜖=0.2: 100%|██████████| 313/313 [08:17<00:00,  1.36s/it]


PGD 𝜖 = 0.2: Accuracy after attack: 0.06%

C-W Attack:


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchattacks import FGSM, PGD, CW
from tqdm import tqdm

from wideresnet import WideResNet

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define WideResNet hyperparameters
depth = 28
width = 10
num_classes = 10
batch_size = 32  # Reduced batch size to minimize memory usage

# Define data transformations and load the dataset
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

test_dataset = datasets.CIFAR10(root='/content/data', train=False, download=True, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained WideResNet model
def load_wideresnet_model(model_path, depth, width, num_classes):
    model = WideResNet(depth=depth, width=width, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

# Replace this with the path to your pre-trained model weights
model_path = '/content/drive/MyDrive/best_wrn_model.pth'
model = load_wideresnet_model(model_path, depth, width, num_classes)

# Function to test the model's accuracy
def test_model_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc='Evaluating Clean Accuracy', leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Clear CUDA cache
torch.cuda.empty_cache()

# Test clean model accuracy
print('Model accuracy on clean test data:')
test_model_accuracy(model, test_loader)

# Attack Functions
def add_noise(images, noise_factor):
    noisy_images = images + noise_factor * torch.randn_like(images)
    return torch.clamp(noisy_images, 0, 1)

def attack_fgsm(model, images, labels, epsilon):
    images.requires_grad = True
    attack = FGSM(model, eps=epsilon)
    return attack(images, labels)

def attack_pgd(model, images, labels, epsilon, alpha, steps):
    images.requires_grad = True
    attack = PGD(model, eps=epsilon, alpha=alpha, steps=steps)
    return attack(images, labels)

def attack_cw(model, images, labels):
    images.requires_grad = True
    attack = CW(model, c=1e-4, kappa=0, steps=1000, lr=0.01)
    return attack(images, labels)

# Function to evaluate model on adversarial examples
def evaluate_adversarial_examples(model, data_loader, attack_fn, description='Attacking'):
    model.eval()
    correct = 0
    total = 0
    for images, labels in tqdm(data_loader, desc=description, leave=False):
        images, labels = images.to(device), labels.to(device)
        adv_images = attack_fn(model, images, labels)
        outputs = model(adv_images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Compare performance drops for different noise magnitudes
def compare_performance_drops(model, data_loader):
    noise_epsilons = [0.05, 0.1, 0.2]
    fgsm_epsilons = [0.05, 0.1, 0.2]
    pgd_epsilons = [0.05, 0.1, 0.2]
    num_steps = 40
    step_size = 0.01

    # print('\nFGSM Attack:')
    # for eps in tqdm(fgsm_epsilons, desc='FGSM Attack', leave=False):
    #     accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: attack_fgsm(model, x, y, epsilon=eps), description=f'FGSM 𝜖={eps}')
    #     print(f'FGSM 𝜖 = {eps}: Accuracy after attack: {accuracy:.2f}%')

    # print('\nPGD Attack:')
    # for eps in tqdm(pgd_epsilons, desc='PGD Attack', leave=False):
    #     accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: attack_pgd(model, x, y, epsilon=eps, alpha=step_size, steps=num_steps), description=f'PGD 𝜖={eps}')
    #     print(f'PGD 𝜖 = {eps}: Accuracy after attack: {accuracy:.2f}%')

    # print('\nC-W Attack:')
    # accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: attack_cw(model, x, y), description='C-W Attack')
    # print(f'C-W attack: Accuracy after attack: {accuracy:.2f}%')

    print('\nNoise Attack:')
    for eps in tqdm(noise_epsilons, desc='Noise Attack', leave=False):
        accuracy = evaluate_adversarial_examples(model, data_loader, lambda model, x, y: add_noise(x, eps), description=f'Noise 𝜖={eps}')
        print(f'Noise magnitude 𝜖 = {eps}: Accuracy after attack: {accuracy:.2f}%')

# Test performance drop comparison
compare_performance_drops(model, test_loader)


Files already downloaded and verified
Model accuracy on clean test data:


Accuracy: 79.48%

Noise Attack:


Noise Attack:  33%|███▎      | 1/3 [00:08<00:16,  8.06s/it]

Noise magnitude 𝜖 = 0.05: Accuracy after attack: 34.80%



Noise Attack:  67%|██████▋   | 2/3 [00:16<00:08,  8.06s/it]

Noise magnitude 𝜖 = 0.1: Accuracy after attack: 30.03%



Noise 𝜖=0.2:  99%|█████████▉| 310/313 [00:07<00:00, 39.75it/s]
                                                           

Noise magnitude 𝜖 = 0.2: Accuracy after attack: 18.46%


In [ ]:
torch.cuda.empty_cache()


In [ ]:
!nvidia-smi

Thu May  9 03:28:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              48W / 400W |  40335MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchattacks import PGD
from tqdm import tqdm

from wideresnet import WideResNet
model_path = '/content/drive/MyDrive/best_wrn_model.pth'

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define WideResNet hyperparameters
depth = 28
width = 10
num_classes = 10
batch_size = 1024
num_epochs = 10
learning_rate = 0.001

# Define data transformations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='/content/data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='/content/data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained WideResNet model
def load_wideresnet_model(model_path, depth, width, num_classes):
    model = WideResNet(depth=depth, width=width, num_classes=num_classes)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model

# Define WideResNet model
model = WideResNet(depth=depth, width=width, num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

# Adversarial Training
def train_adversarial(model, train_loader, optimizer, criterion, device, attack_fn):
    model.train()
    for images, labels in tqdm(train_loader, desc='Adversarial Training', leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        # Create adversarial examples
        adv_images = attack_fn(model, images, labels)
        # Compute loss on adversarial examples
        outputs = model(adv_images)
        loss = criterion(outputs, labels)
        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

# Define PGD Attack for Adversarial Training
def attack_pgd_training(model, images, labels):
    attack = PGD(model, eps=0.03, alpha=0.01, steps=40)
    return attack(images, labels)

# Training Loop
def train_model(model, train_loader, test_loader, num_epochs, optimizer, criterion):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        train_adversarial(model, train_loader, optimizer, criterion, device, attack_pgd_training)
        test_model_accuracy(model, test_loader)

# Test model accuracy function
def test_model_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc='Evaluating Clean Accuracy', leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Clear CUDA cache
torch.cuda.empty_cache()

# Train model using adversarial training
train_model(model, train_loader, test_loader, num_epochs, optimizer, criterion)

# Save the adversarially trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/adversarially_trained_wrn_model.pth')

# Test performance after adversarial training
print('Model accuracy on clean test data after adversarial training:')
test_model_accuracy(model, test_loader)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10


Accuracy: 15.50%
Epoch 2/10


Accuracy: 15.15%
Epoch 3/10


Adversarial Training:  82%|████████▏ | 40/49 [09:15<02:05, 13.89s/it]